In [21]:
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Optimizer

from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange

import scipy.io
from utils import log_gaussian_loss, gaussian, get_kl_Gaussian_divergence

from collections import OrderedDict

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device: {}'.format(device))

device: cuda


In [22]:
class DNN(nn.Module):
    def __init__(self, layers):
        super().__init__()

        n_hidden = len(layers) 
        self.activation = nn.Tanh
        self.dropout = nn.Dropout
        layer_list = []

        for i in range(n_hidden-2):

            linear = torch.nn.Linear(layers[i], layers[i+1])

            nn.init.xavier_normal_(linear.weight.data, gain=1.0)
            nn.init.zeros_(linear.bias.data)

            layer_list.append(
                ('layer_%d' % i, linear)
            )
            layer_list.append(
                ('activation_%d' % i, self.activation())
            )

            layer_list.append(
                ('dropout_%d' % i, self.dropout(p = 0.2))
            )
        
        linear = torch.nn.Linear(layers[n_hidden-2], layers[n_hidden-1])
        nn.init.xavier_normal_(linear.weight.data, gain=1.0)
        nn.init.zeros_(linear.bias.data)

        layer_list.append(('layer_%d' % (n_hidden-2), linear))
        
        layerDict = OrderedDict(layer_list)
        self.layers = torch.nn.Sequential(layerDict)

    def forward(self, x):
        out = self.layers(x)
        return out

In [23]:
class PINN_Burgers:
    
    def __init__(self, net, Xt, Y, lb, ub, device):
        
        # data
        self.X = Xt[:,0:1]; self.t = Xt[:,1:2]
        self.Y = torch.tensor(Y).float().to(device)
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)

        # net
        self.net = net.to(device)
        
        self.lambda_1 = torch.tensor([0.0], requires_grad=True).to(device)
        self.lambda_2 = torch.tensor([-6.0], requires_grad=True).to(device)
        self.lambda_1 = torch.nn.Parameter(self.lambda_1)
        self.lambda_2 = torch.nn.Parameter(self.lambda_2)
        self.net.register_parameter('lambda_1', self.lambda_1)
        self.net.register_parameter('lambda_2', self.lambda_2)

       
        self.iter = 0
        self.opt_Adam =  torch.optim.AdamW(self.net.parameters(), lr = 0.003)
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.opt_Adam, max_lr = 1e-2, 
                                            steps_per_epoch = 1, epochs = 10000)

    def net_U(self, x, t):
        xt = torch.cat((x,t), dim=1)
        xt = 2*(xt-self.lb)/(self.ub-self.lb) - 1
        u = self.net(xt)
        return u

    def net_F(self, x, t):
        lambda_1 = self.lambda_1        
        lambda_2 = torch.exp(self.lambda_2)

        u = self.net_U(x, t)

        u_t = torch.autograd.grad(u, t, torch.ones_like(u),
                                    retain_graph=True,
                                    create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, torch.ones_like(u), 
                                    retain_graph=True,
                                    create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x),
                                    retain_graph=True,
                                    create_graph=True)[0]
      
        F = u_t + lambda_1*u*u_x - lambda_2*u_xx
        
        return F

    def _train_step(self):
        
        X = torch.tensor(self.X, requires_grad=True).float().to(device)
        t = torch.tensor(self.t, requires_grad=True).float().to(device)
        
        u_pred = self.net_U(X, t)
        f_pred = self.net_F(X, t)
        loss = torch.mean((self.Y-u_pred)**2) + torch.mean(f_pred**2)

        return loss

    def _closure(self):
        loss = self._train_step()
        self.opt_LBFGS.zero_grad()
        loss.backward()

        self.iter += 1
        if self.iter % 200 == 0:
            print('epoch: {}  loss: {:.3e}  lam_1: {:4f}  lam_2: {:.6f}'.format(self.iter, 
                                                                                loss.item(),
                                                                                self.lambda_1.item(),
                                                                                torch.exp(self.lambda_2).item()))
        return loss

    def train(self, epochs):
        self.net.train() # training mode
        for epoch in range(epochs):
            
            loss = self._train_step() 
            
            self.opt_Adam.zero_grad()
            loss.backward()
            self.opt_Adam.step()

            if (epoch+1) % 200 == 0:
                print('epoch: {}  loss: {:.3e}  lam_1: {:4f}  lam_2: {:.6f}'.format(epoch+1, 
                                                                                    loss.item(),
                                                                                    self.lambda_1.item(),
                                                                                    torch.exp(self.lambda_2).item()))
     

    def predict(self, x):
        
        x = torch.tensor(x, requires_grad=True).float().to(device)
    
        self.net.eval()
       
        u_pred = self.net_U(x[:,0:1], x[:,1:2])
        
        return u_pred.detach().cpu().numpy()
        




In [24]:
N_u = 2000
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

import scipy.io
data = scipy.io.loadmat('./Data/burgers_shock.mat')

t = data['t'].flatten()[:,None] # 100 x 1
x = data['x'].flatten()[:,None] # 256 x 1
Exact = np.real(data['usol']).T # 100 x 256

X, T = np.meshgrid(x,t) # 100 x 256

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None])) # 25600 x 2
u_star = Exact.flatten()[:,None] # 25600 x 1         

# Domain bounds of x, t
lb = X_star.min(0)
ub = X_star.max(0)   

# training data
noise = 0.0            
idx = np.random.choice(X_star.shape[0], N_u, replace=False)
X_u_train = X_star[idx,:]
u_train = u_star[idx,:]

In [25]:
net = DNN(layers)
net

DNN(
  (layers): Sequential(
    (layer_0): Linear(in_features=2, out_features=20, bias=True)
    (activation_0): Tanh()
    (dropout_0): Dropout(p=0.2, inplace=False)
    (layer_1): Linear(in_features=20, out_features=20, bias=True)
    (activation_1): Tanh()
    (dropout_1): Dropout(p=0.2, inplace=False)
    (layer_2): Linear(in_features=20, out_features=20, bias=True)
    (activation_2): Tanh()
    (dropout_2): Dropout(p=0.2, inplace=False)
    (layer_3): Linear(in_features=20, out_features=20, bias=True)
    (activation_3): Tanh()
    (dropout_3): Dropout(p=0.2, inplace=False)
    (layer_4): Linear(in_features=20, out_features=20, bias=True)
    (activation_4): Tanh()
    (dropout_4): Dropout(p=0.2, inplace=False)
    (layer_5): Linear(in_features=20, out_features=20, bias=True)
    (activation_5): Tanh()
    (dropout_5): Dropout(p=0.2, inplace=False)
    (layer_6): Linear(in_features=20, out_features=20, bias=True)
    (activation_6): Tanh()
    (dropout_6): Dropout(p=0.2, inplace

In [26]:
pinn = PINN_Burgers(net, X_u_train, u_train, lb, ub, device)
pinn.train(0) 

AttributeError: 'PINN_Burgers' object has no attribute 'opt_LBFGS'